In [2]:
import folium
import random
import numpy as np

# Coordenadas iniciais de BH (Belo Horizonte)
map_center = [-19.9208, -43.9378]

# Função para gerar coordenadas aleatórias espaçadas em torno de um ponto central
def generate_random_location(center, spread=0.05):
    return [center[0] + random.uniform(-spread, spread), center[1] + random.uniform(-spread, spread)]

# Gerar coordenadas aleatórias espaçadas para alunos
students = [{'name': f'Aluno {i+1}', 'location': generate_random_location(map_center, spread=0.03)} for i in range(6)]

# Gerar coordenadas aleatórias espaçadas para escolas
schools = [{'name': f'Escola {chr(65+i)}', 'location': generate_random_location(map_center, spread=0.03)} for i in range(2)]

# Gerar coordenadas aleatórias espaçadas para garagens
garages = [{'name': f'Garagem {i+1}', 'location': generate_random_location(map_center, spread=0.03)} for i in range(2)]

# Função de Cálculo de Distância
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Função de Fitness
def fitness(route):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += calculate_distance(route[i], route[i + 1])
    return total_distance

# Função de Criação de Rotas
def create_route(garage, students, school):
    student_locations = [student['location'] for student in students]
    random.shuffle(student_locations)
    stops = [garage['location']] + student_locations + [school['location']] + [garage['location']]
    return stops

# Função de Mutação
def mutate(route):
    student_positions = route[1:-3]  # Alunos estão entre a primeira garagem e a escola
    idx1, idx2 = random.sample(range(len(student_positions)), 2)
    student_positions[idx1], student_positions[idx2] = student_positions[idx2], student_positions[idx1]
    return [route[0]] + student_positions + route[-3:]

# Algoritmo Genético
def genetic_algorithm(garage, students, school, population_size=100, generations=500, mutation_rate=0.1):
    population = [create_route(garage, students, school) for _ in range(population_size)]
    best_route = min(population, key=fitness)
    
    for generation in range(generations):
        new_population = []
        for route in population:
            if random.random() < mutation_rate:
                new_route = mutate(route[:])
                new_population.append(new_route)
            else:
                new_population.append(route)
        
        best_route_in_population = min(new_population, key=fitness)
        if fitness(best_route_in_population) < fitness(best_route):
            best_route = best_route_in_population
        
        population = new_population
    
    return best_route

# Definir os alunos para cada escola
students_school_A = students[:3]
students_school_B = students[3:]

# Gerar rotas otimizadas
best_route_A = genetic_algorithm(garages[0], students_school_A, schools[0])
best_route_B = genetic_algorithm(garages[1], students_school_B, schools[1])

# Definir a lista de cores variando do vermelho ao violeta
color_scale = ['red', 'orange', 'green', 'lightgreen', 'cadetblue', 'blue', 'darkblue', 'purple', 'darkpurple', 'pink', 'lightred', 'beige', 'lightgray', 'gray', 'white', 'lightblue', 'lightpurple', 'darkred', 'black']

# Criar um novo mapa para adicionar as rotas
school_map_with_routes = folium.Map(location=map_center, zoom_start=12)

# Adicionar marcadores para alunos no novo mapa
for student in students:
    folium.Marker(location=student['location'], tooltip=student['name'], icon=folium.Icon(color='blue', icon='info-sign')).add_to(school_map_with_routes)

# Adicionar marcadores para escolas no novo mapa
for school in schools:
    folium.Marker(location=school['location'], tooltip=school['name'], icon=folium.Icon(color='green', icon='school')).add_to(school_map_with_routes)

# Adicionar marcadores para garagens no novo mapa
for garage in garages:
    folium.Marker(location=garage['location'], tooltip=garage['name'], icon=folium.Icon(color='red', icon='cog')).add_to(school_map_with_routes)

# Adicionar linhas representando as rotas no mapa
folium.PolyLine(best_route_A, color='red', weight=2.5, opacity=1).add_to(school_map_with_routes)
folium.PolyLine(best_route_B, color='blue', weight=2.5, opacity=1).add_to(school_map_with_routes)

# Adicionar marcadores para os pontos de parada das rotas no mapa com cores variadas
for i, stop in enumerate(best_route_A):
    folium.Marker(location=stop, tooltip=f'Parada {i+1}', icon=folium.Icon(color=color_scale[i % len(color_scale)], icon='flag')).add_to(school_map_with_routes)

for i, stop in enumerate(best_route_B):
    folium.Marker(location=stop, tooltip=f'Parada {i+1}', icon=folium.Icon(color=color_scale[i % len(color_scale)], icon='flag')).add_to(school_map_with_routes)

# Adicionar legenda ao mapa
legend_html = '''
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 150px; height: 150px; 
    border:2px solid grey; z-index:9999; font-size:14px;
    background-color:white;
    ">
    <b>&nbsp;Legenda</b><br>
    &nbsp;<i class="fa fa-info-sign" style="color:blue"></i>&nbsp;Aluno<br>
    &nbsp;<i class="fa fa-school" style="color:green"></i>&nbsp;Escola<br>
    &nbsp;<i class="fa fa-cog" style="color:red"></i>&nbsp;Garagem<br>
    &nbsp;<i class="fa fa-flag" style="color:orange"></i>&nbsp;Paradas Rota A<br>
    &nbsp;<i class="fa fa-flag" style="color:purple"></i>&nbsp;Paradas Rota B<br>
</div>
'''
school_map_with_routes.get_root().html.add_child(folium.Element(legend_html))

# Adicionar grid ao mapa usando JavaScript
grid_js = '''
<script>
L.GridLayer.DebugCoords = L.GridLayer.extend({
    createTile: function (coords) {
        var tile = document.createElement('div');
        tile.innerHTML = [coords.x, coords.y, coords.z].join(', ');
        tile.style.outline = '1px solid red';
        return tile;
    }
});

L.gridLayer.debugCoords = function(opts) {
    return new L.GridLayer.DebugCoords(opts);
}

L.gridLayer.debugCoords().addTo(map);
</script>
'''

school_map_with_routes.get_root().html.add_child(folium.Element(grid_js))

# Salvar o mapa com rotas em um arquivo HTML
school_map_with_routes.save('school_routes_map.html')

# Exibir o mapa com rotas no Jupyter Notebook (se estiver usando Jupyter)
school_map_with_routes
